In [1]:
# import stats packages
import numpy as np
import pandas as pd
import sys, os
import glob
import fiona
from plotnine import ggplot

# import spatial modules
import ee
import geemap
#from osgeo import gdal #install with pip3, don't use conda

# don't display warnings
import warnings
warnings.filterwarnings('ignore') 

In [2]:
#********************************* Initialize GEE  ***********************************#
## trigger the authentication flow. only need once
ee.Authenticate()
## After inserting the API key initialize GEE
ee.Initialize()

Enter verification code: 4/1ARtbsJoZRtdTZa7wtCW3DNsm7hp3_t-Ppk471D8CkUCAt_NVwHkH2Le0IoU

Successfully saved authorization token.


In [6]:
#****************************** define user parameters *******************************#
# setup output directory
outDIR="/Users/darylyang/Desktop/conda/gee/test"
# create output folder if it does not exist
os.makedirs(outDIR, exist_ok=True)
# print output directory
print(outDIR)

# setup date range in which MODIS data will be extracted
dateBEG = ee.Date("2011-01-01")
dateEND = ee.Date("2015-12-31")
# setup month in which MODIS data will be extracted 
monBEG = 7
monEND = 7

# set visualization parameters.
lst_vis_params = {
    'min': 100,
    'max': 300,
    'palette': ['21325E', '662A00', 'F6F54D']
}

/Users/darylyang/Desktop/conda/gee/test


In [7]:
#********************************* load in data **************************************#
# load in modis land surface albedo
modCOLL = ee.ImageCollection('MODIS/006/MCD43A3').filter(ee.Filter.date(dateBEG, dateEND))
modCOLL = modCOLL.filter(ee.Filter.calendarRange(monBEG,monEND,'month'));
# count the number of images found
count = modCOLL.size()
print('Count: ', str(count.getInfo())+'\n')
# extract white sky and black sky albedo
modALB_WSA = modCOLL.select('Albedo_WSA_shortwave')
modALB_BSA = modCOLL.select('Albedo_BSA_shortwave')

# load in a polygon where MODIS will be clipped out
shpDIR = "/Volumes/data2/dyang/projects/arctic_energy/boundary/above_domain_use.shp"
shpVCT = geemap.shp_to_ee(shpDIR)

Count:  155



In [8]:
#******************************** preprocessing *************************************#
# clip out the region defined by shp file
def geeclip (image):
    return image.clip(shpVCT)
modALB_WSA_CLP = modALB_WSA.map(geeclip)
modALB_BSA_CLP = modALB_BSA.map(geeclip)

# calculate mean
modALB_WSA_MN = modALB_WSA_CLP.reduce(ee.Reducer.mean())
modALB_BSA_MN = modALB_BSA_CLP.reduce(ee.Reducer.mean())

modALB_MN = modALB_WSA_MN.add(modALB_BSA_MN).divide(2)

# make a quick display show the data region
Map = geemap.Map(center=(65, -120), zoom=3)
Map.addLayer(shpVCT, {}, 'ROI')
Map.addLayer(modALB_MN, lst_vis_params, 'MODIS LST', True, 1)
Map.add_colorbar(vis_params=lst_vis_params, label='MODIS LST (Scale = 0.001)')
Map

Map(center=[65, -120], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [43]:
#******************************** preprocessing *************************************#
geometry = ee.Geometry.Rectangle([-168, 49, -81, 80])
#geometry = shpVCT.geometry().bounds().getInfo()['coordinates']
task = ee.batch.Export.image.toDrive(**{
    'image': modALB_MN, 
    'description': 'modis_albedo_above_month_12', 
    'folder': 'modis_albedo', 
    'scale': 500, 
    'region': geometry, #shpVCT.geometry().bounds().getInfo()['coordinates'], 
    'maxPixels': 10e12 })
task.start()